In [9]:
import os
import time

import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import re
import matplotlib

from nltk.stem.snowball import SnowballStemmer
import nltk
import gensim
import spacy

import matplotlib.cm as cm

stemmer = SnowballStemmer("english")
nlp = spacy.load("en_core_web_sm")

In [10]:
from tool import *
# unique(x)
# find_kol(data,rank)
# unique_time(data)
# exforeign(data)
# duplicate(data)
# remove_link(data)
# remove_sign(data)
# remove_rubbish(data)
# total_clean(indata)
# toword(data,stem=False,stop=True) # return list of list
# get_words(url), pos, neg, opinion
# debug2(sentence)
# month(start,end)
# search(data,keywords)
# text(data)
def find_user(data,screen_name):
    result=[]
    k=None
    for i in data:
        if i['screen_name']==screen_name:
            result.append(i['text'])
            k=i
    return result,i

In [11]:
otherwords={'hi','wealthfront','betterment','robo','roboadvisor','roboadvisors',
            'wealthfronts','happy','im','vs','thanks','like','love','yet','yeah','via','want','one','a','an','advisor',
            'advisors','dont','didnt','financial','investment','wealth','finance'}

otherwords=otherwords.union(set(gensim.parsing.preprocessing.STOPWORDS))
def LDA(data, topics, words,otherwords):
    import gensim
    from gensim import corpora
    from gensim.models.ldamodel import LdaModel
    texts=toword(data,stem=False,lemma=True,stop=True,otherwords=otherwords)
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]
    
    lda = LdaModel(corpus,id2word=dictionary,num_topics=topics,passes=20)
    temp=lda.print_topics(num_words=words)
    return [i[1] for i in temp]

def NMF(data,topics,words,otherwords):
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.decomposition import NMF
    string=[' '.join(i) for i in toword(data,stem=False,lemma=True,stop=True,otherwords=otherwords)]
    import gensim
    STOPWORDS= set(nltk.corpus.stopwords.words('english')).union(otherwords)
    tfidf_vect = TfidfVectorizer(max_df=0.8, min_df=2, stop_words=STOPWORDS)
    doc_term_matrix = tfidf_vect.fit_transform(string)
    nmf = NMF(n_components=topics)
    nmf.fit(doc_term_matrix)
    
    return [' + '.join(["\"{}\"".format(tfidf_vect.get_feature_names()[i]) for i in topic.argsort()[-words:]]) for topic in nmf.components_]

In [4]:
data0=json.load(open('./data/Wealthfront/Wealthfront2017-06.json'))
print(len(data0))
data=total_clean(data0)
print(len(data))

1945
371


In [5]:
LDA(data,10,7,otherwords)

['0.014*"new" + 0.013*"factor" + 0.010*"redesign" + 0.008*"client" + 0.008*"week" + 0.008*"passive" + 0.006*"open"',
 '0.016*"indexing" + 0.013*"fund" + 0.010*"advanced" + 0.010*"market" + 0.010*"personal" + 0.010*"capital" + 0.007*"cost"',
 '0.014*"use" + 0.011*"work" + 0.008*"people" + 0.008*"know" + 0.008*"thats" + 0.008*"come" + 0.008*"straying"',
 '0.016*"smart" + 0.016*"beta" + 0.011*"year" + 0.008*"know" + 0.008*"help" + 0.008*"best" + 0.008*"design"',
 '0.016*"fee" + 0.016*"link" + 0.013*"friend" + 0.010*"free" + 0.010*"posted" + 0.008*"trying" + 0.007*"rachleff"',
 '0.025*"great" + 0.016*"look" + 0.010*"account" + 0.010*"week" + 0.006*"guy" + 0.006*"marketing" + 0.006*"index"',
 '0.061*"andy" + 0.059*"rachleff" + 0.048*"growing" + 0.046*"corner" + 0.045*"founder" + 0.035*"business" + 0.010*"better"',
 '0.013*"hard" + 0.013*"strategy" + 0.013*"customer" + 0.009*"business" + 0.009*"good" + 0.009*"blog" + 0.009*"work"',
 '0.050*"feel" + 0.037*"human" + 0.034*"trying" + 0.023*"loo

In [20]:
text(search(data,['beta']))

['Burton Malkiel is straying from his Gospel and starts promoting Smart Beta Macht ja Wealthfront auch ',
 ' has in effect been expensive beta Burt Malkiel presents the approach to Smart Beta ',
 'Why Wealthfronts Smart Beta ',
 'Malkiels Smart Beta critique was deleted from the WealthFront website ',
 'Wealthfront macht aber gerade erst Smart Beta ',
 'Smart beta gets an important stamp of approval Burt Malkiel but only when fees are low ',
 'But now youre claiming youve found the Smart Beta strategy that solves the performance problem also This is a bad look',
 'Strange to see WealthFront and other Robos pivoting towards Smart Beta after trashing it for years',
 'With Advanced Indexing multifactor smart beta overlay Wealthfront Pivots To Embrace Smart Beta via ',
 '1Wow Wealthfront turns to smart beta ',
 'Interesting but needs more detail on how its different from other smart beta products',
 'Why is now embracing smart beta strategies ',
 'Wonder what thinks about smart beta',
 'In

In [13]:
root_path=os.getcwd()
category=os.listdir(root_path+'\\data\\')

In [14]:
m=month('2017-06','2020-05')
metadata=[]
for i in m:
    temp=[]
    for j in ['Betterment','Wealthfront','RoboAdvisor',]:
        try:
            temp+=json.load(open(root_path+'\\data\\{}\\{}{}.json'.format(j,j,i)))
        except:
            pass
    metadata.append(total_clean(temp))

In [15]:
len(metadata[0])

1114

In [22]:
LDA(search(metadata[0],['beta']),10,7,otherwords)

['0.080*"macht" + 0.080*"gerade" + 0.080*"erst" + 0.080*"aber" + 0.080*"smart" + 0.080*"beta" + 0.007*"think"',
 '0.013*"smart" + 0.013*"beta" + 0.013*"different" + 0.013*"think" + 0.013*"embracing" + 0.013*"bad" + 0.013*"20"',
 '0.116*"beta" + 0.116*"smart" + 0.048*"indexing" + 0.048*"advanced" + 0.025*"solves" + 0.025*"problem" + 0.025*"look"',
 '0.142*"beta" + 0.142*"smart" + 0.032*"burt" + 0.032*"important" + 0.032*"get" + 0.032*"fee" + 0.032*"approval"',
 '0.129*"smart" + 0.129*"beta" + 0.051*"malkiel" + 0.027*"burton" + 0.027*"promoting" + 0.027*"straying" + 0.027*"start"',
 '0.074*"bad" + 0.074*"said" + 0.074*"2015" + 0.074*"quotable" + 0.074*"stupid" + 0.074*"smart" + 0.074*"beta"',
 '0.080*"website" + 0.080*"critique" + 0.080*"malkiels" + 0.080*"deleted" + 0.080*"beta" + 0.080*"smart" + 0.007*"strategy"',
 '0.059*"strategy" + 0.059*"panel" + 0.059*"expert" + 0.059*"20" + 0.059*"jun" + 0.059*"comparison" + 0.059*"tue"',
 '0.013*"beta" + 0.013*"smart" + 0.013*"interesting" + 0.0

In [9]:
text(search(metadata[0],['Blackrock','blackrock']))

['Scalable Capital with 250 million in assets 6000 retail clients Raises 33 Million from BlackRock ',
 'Roboadvisors I was shocked When saw this yesterday 90 vs 10 especially that BlackRock will use it Should we consider it',
 'BlackRock Invests in European RoboAdvisor ',
 'Curated on BlackRock installing roboadvisors to replace humans as ETF stockpickers ',
 'BlackRock Takes Stake In RoboAdvisor Scalable Capital BLK',
 'Scalable Capital Blackrock steigt beim Roboadvisor ein manager magazin ',
 'BlackRock makes a huge investment in European ',
 'BlackRock takes Scalable Capital stake in Europe roboadvisor push via ',
 'BlackRock takes Scalable Capital stake in Europe roboadvisor push Reuters ',
 'As Blackrock invests 30mil in Repost I am with French when to ',
 ' BlackRock confirme son ticket dans le roboadvisor Scalable Capital via ',
 'NEW BlackRocks Global Investor Pulse Survey ',
 'BlackRock takes Scalable Capital stake in Europe roboadvisor push ',
 'BlackRock Takes Scalable Capit

In [10]:
LDA(metadata[1],10,7,otherwords)

['0.018*"sector" + 0.016*"ready" + 0.012*"new" + 0.012*"5" + 0.012*"rise" + 0.011*"800m" + 0.011*"willing"',
 '0.036*"investing" + 0.017*"transform" + 0.015*"roboanalysts" + 0.012*"know" + 0.012*"digital" + 0.011*"client" + 0.011*"mortgage"',
 '0.020*"future" + 0.012*"portfolio" + 0.010*"fee" + 0.010*"know" + 0.009*"think" + 0.009*"advisory" + 0.007*"stock"',
 '0.020*"investor" + 0.015*"question" + 0.015*"ask" + 0.012*"read" + 0.009*"new" + 0.009*"fee" + 0.009*"invest"',
 '0.020*"change" + 0.013*"advisory" + 0.011*"experience" + 0.011*"build" + 0.010*"investing" + 0.010*"starting" + 0.010*"fund"',
 '0.030*"rt" + 0.016*"management" + 0.010*"traditional" + 0.009*"money" + 0.009*"report" + 0.009*"challenged" + 0.009*"boom"',
 '0.017*"investing" + 0.016*"use" + 0.010*"automated" + 0.010*"market" + 0.010*"major" + 0.010*"platform" + 0.008*"13"',
 '0.025*"million" + 0.017*"wealthsimple" + 0.016*"review" + 0.013*"800" + 0.013*"valuation" + 0.012*"uk" + 0.011*"raise"',
 '0.023*"human" + 0.018*